In [ ]:
#Alternate Data Sources: https://www.equibase.com/
# http://horseracingdatasets.com/


In [ ]:
# https://www.reddit.com/r/MachineLearning/comments/8jnzwt/d_horse_race_winner_prediction_89_precision_80/
# https://www.youtube.com/watch?v=5gW0PO7g6pY
# https://www.youtube.com/watch?v=6tQhoUuQrOw

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df_horses = pd.read_csv("horses.csv")

c:\users\jeff\appdata\local\programs\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_horses.head()
# What is the target column, what are we trying to predict? 
# Every row is a horse, es

,R,TB,AQU,20181114,1,D,Claiming,36000,"FOR THREE YEAR OLDS AND UPWARD WHICH HAVE NEVER WON TWO RACES. Three Year Olds, 120 lbs.; Older, 123 lbs. Non-winners of a","race at a mile or over since May 1 Allowed 2 lbs. Claiming Price $25,000.",...,Unnamed: 76,0.10,0.11,Unnamed: 79,Unnamed: 80,0.12,0.13,"MAGNETRON;25000; Windy Lea Farms,Bruce Brown;",Unnamed: 84,0.14
0,H,AQU,20181114,1.0,D,26Oct18 9BEL12,TB,Won't Be Missed,110,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,AQU,20181114,1.0,D,22Sep18 12PRX 6,TB,Purchasing Power,120,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H,AQU,20181114,1.0,D,14Oct18 6BEL 6,TB,Malibu Strings,118,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H,AQU,20181114,1.0,D,13Oct18 3BEL 1,TB,Magnetron,120,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,AQU,20181114,1.0,D,30Sep18 4PRX 6,TB,Incubator,118,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_horses.shape

(221680, 108)

In [5]:
df_horses = df_horses.dropna()

In [6]:
df_horses.head()

,previous_margin,position,position_again,position_two,margin,bf_odds,bf_odds_two_mins_out,bf_odds_all,vic_tote,vic_tote_two_mins_out,...,class_stronger_places,first_up_starts,first_up_wins,first_up_places,second_up_starts,second_up_wins,second_up_places,track_distance_starts,track_distance_wins,track_distance_places
0,6.8,1,1.0,1.0,0.0,2.88,3.0,"{3.25,3.05,3.0,2.92,2.88}",2.3,3.5,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,1,1.0,1.0,0.0,4.90,5.3,"{5.8,5.9,5.9,5.8,5.6,5.3,5.2,5.3,4.9,4.9}",3.6,4.8,...,2.0,5.0,1.0,3.0,5.0,1.0,2.0,4.0,2.0,1.0
65,5.5,1,1.0,1.0,0.0,5.00,4.1,"{4.6,4.5,4.5,4.6,4.7,4.7,5.1,4.1,4.8,5.0}",4.4,5.4,...,2.0,7.0,0.0,2.0,6.0,0.0,1.0,2.0,2.0,0.0
73,5.9,1,1.0,1.0,0.0,8.80,7.8,"{5.4,5.4,6.2,8.0,8.4,8.0,8.0,7.8,8.6,8.8}",8.3,5.9,...,2.0,3.0,2.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0
151,5.6,1,1.0,1.0,0.0,21.00,18.0,"{12.5,13.0,13.5,17.0,15.0,15.5,17.5,18.0,19.5,...",10.8,10.9,...,0.0,2.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0


In [7]:
df_min = df_horses[['position','condition', 'runner_id', 'barrier', 'sex']]

In [8]:
df_min.head()

,position,condition,runner_id,barrier,sex
0,1,HVY9,2,3.0,Gelding
51,1,HVY10,77,4.0,Gelding
65,1,HVY10,92,6.0,Gelding
73,1,HVY9,110,2.0,Gelding
151,1,GOOD4,444,3.0,Mare


In [11]:
df_min.head()
#Get target column
target_df = df_min['position']
df_min = df_min.drop('position', 1)

In [12]:
features_final = pd.get_dummies(df_min)

In [13]:
features_final.shape


(27367, 20)

In [14]:
from sklearn.model_selection import train_test_split
# Parameters are features, target, split ratio, random seed
X_train, X_test, y_train, y_test = train_test_split(features_final, target_df ,test_size=0.20, random_state=42)

In [22]:
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 21893 samples.
Testing set has 5474 samples.


In [35]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)


In [40]:
from sklearn.metrics import fbeta_score, accuracy_score
from time import time
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end-start
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    #print (type(predictions_test))
    #print (predictions_train)
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
    
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train)
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = fbeta_score(y_train[:300], predictions_train, 0.5)
        
    # TODO: Compute F-score on the test set which is y_test
    # 3rd param is Beta
    results['f_test'] = fbeta_score(y_test, predictions_test, 0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results


In [42]:
#print(y_train.size)
samples_100 = y_train.size
train_predict(clf, samples_100, X_train, y_train, X_test, y_test)

RandomForestClassifier trained on 21893 samples.


{'acc_test': 0.68012422360248448,
 'acc_train': 0.70666666666666667,
 'f_test': 0.63316993464052296,
 'f_train': 0.68904593639575973,
 'pred_time': 0.06996011734008789,
 'train_time': 0.945467472076416}

In [44]:
#Feature importance
model = clf.fit(X_train[:samples_100], y_train[:samples_100])

In [45]:
importances = model.feature_importances_

In [46]:
importances

array([  9.04494112e-01,   5.26681630e-02,   3.06971255e-03,
         7.46674341e-04,   6.48308288e-04,   4.80156892e-03,
         4.58022761e-03,   2.37110896e-05,   4.12544144e-03,
         2.13878854e-03,   1.50566884e-03,   2.15269109e-03,
         1.26949341e-03,   2.50399204e-03,   2.33202587e-03,
         2.89862317e-03,   3.40797876e-03,   2.19790126e-03,
         4.38888255e-03,   4.60354735e-05])

In [ ]:
#Scatter Matrix